# Distribute Python software

## Introduction

### Objectives

Build, install, package and distribute Python code

References:

* https://setuptools.readthedocs.io/en/latest/
* https://packaging.python.org/tutorials/packaging-projects/
* https://github.com/pypa/sampleproject

### Packaging

Packages are the best way to distribute a library, regardless of the operating system.
For (graphical) applications, "fat-binaries" may be a better choice, especially under Windows and macOS.

There are different of packages to be distinguished:

* Python specific packages: Wheels
* Operating system packages: RPM, DEB, MSI, ...
* Conda packages: Not discussed today

Advantages of packaging tools:

* Keeps track of installed packages
* Management of dependencies
* Provides access to a package repository.

### Outline

1. Prepare for packaging
   1. Quick start: The bare minimum
   1. Enhance project description
1. Distribution
1. Advanced features
1. A word on automated deployment
1. A word on packages

## Prepare for packaging

Python packaging has a history and legacy.

We present only the most common solution as of today: [setuptools](https://setuptools.pypa.io/en/latest/) and [pip](https://pypi.org/project/pip/) (the **p**ackage **i**nstaller for **P**ython)

Note: There are others: [flit](https://pypi.org/project/flit/), [maturin](https://pypi.org/project/maturin/), [poetry](https://python-poetry.org/docs/).

### First, make a package

Create a directory with the name of your package and a `__init__.py` file in it.

```
project/
    package/
        __init__.py
        module.py
        subpackage/
            __init__.py
        ...
```

Often the *project* and the *package* names are the same.

So that, from the `project` directory:

```python
>>> import package
>>> from package import module
>>> import package.subpackage
```

### Quick start

[setuptools](https://setuptools.pypa.io/en/latest/) and [pip](https://pypi.org/project/pip/) rely on:
- 2 configuration files: `pyproject.toml` and `setup.cfg`
- a `setup.py` script (optional)

#### pyproject.toml

`pyproject.toml` describes project's build dependencies ([PEP 518](https://www.python.org/dev/peps/pep-0518/)).

Sample `pyproject.toml` for using [setuptools](https://setuptools.pypa.io/en/latest/):
```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
```

#### setup.cfg

`setup.cfg` describes the project metadata and build configuration for [setuptools](https://setuptools.pypa.io/en/latest/).
The bare minimum:
```cfg
[metadata]
name = package
version = 0.0.1

[options]
packages = find:  # Automatic package discovery
# Or a list: packages = package, package.subpackage
```

For a single `module.py` file project:

```cfg
[metadata]
name = package
version = 0.0.1

[options]
py_modules = module
```

#### Version number

Stay compatible with Python's "Version Identification and Dependency Specification" [PEP 440](https://www.python.org/dev/peps/pep-0440/#version-scheme):

Some common versioning:
- `major.minor[.micro][{a|b|rc}N]`: 1.0, 1.1.1b1
- `year.month`: 2021.10

#### setup.py

It used to be the central place for building and packaging Python projects (and it still is for many projects), but it is now optional.

Useful for backward compatibility:
```python
# coding: utf-8
import setuptools

if __name__ == "__main__":
    setuptools.setup()
```

It is also the place for defining specific commands and C extensions.

#### Install from source

- Install your package:
  ```
  pip install . [--user]
  ```
- Install in editable mode (aka., develop mode):
  ```
  pip install -e .
  ```
- Previous way:
  ```
  python setup.py install
  ```

#### Build packages

With the [build](https://pypi.org/project/build/) package (`pip install build`):
```
python -m build
```
generates a source tarball and a wheel (`*.whl`) files in `dist/`:
- Source tarball needs to be built before installation
- A wheel file (`.whl`) is a zip file containing an already built Python package
  
Former way (`pip install setuptools wheel`):
```
python setup.py sdist bdist_wheel
```

### Additional information files

Good practice (almost mandatory for distribution):

- `LICENSE`: Contract for using the package
- `README`: Abstract

### Project's description

More information about the project can be added to `setup.cfg` section:

```cfg
[metadata]
```

#### Version

It is simpler to store the version number at a unique place:
- `package/__init__.py`:
  ```python
  ...
  version = "0.0.1"
  ...
    ```
- `setup.cfg`:
  ```cfg
  [metadata]
  name = package
  version = attr: package.version  # Instead of duplicated version number
  ```

#### More metadata
`setup.cfg`:
```cfg
[metadata]
...
# Optional
author = Someone
author_email = someone@somewhere.org
description = Sample project for distribution training
license = MIT
url = https://github.com/silx-kit/silx-training/
project_urls = 
	Bug Tracker = https://github.com/silx-kit/silx-training/issues

```

#### Long description

Reuse project's "README" in project metadata to avoid duplication:

- `setup.cfg`:
  ```cfg
  [metadata]
  ...
  long_description = file: README.md
  long_description_content_type = text/markdown
  ```
- `README.md`:
  ```
  This is a very nice project.
  
  It does a lot of useful things.
  ```

Note: "README" and `*.md` discussed later in the documentation section.

#### Classifiers

```cfg
[metadata]
...
classifiers =
    Development Status :: 3 - Alpha
    Intended Audience :: Education
    License :: OSI Approved :: MIT License
    Operating System :: MacOS
    Operating System :: Microsoft :: Windows
    Operating System :: POSIX
    Programming Language :: Python :: 3
```

Available classifiers: https://pypi.org/classifiers/

### Build configuration 

Build configuration is defined both in the `pyproject.toml` and `setup.cfg` section:

```cfg
[options]
packages = find:
```

#### Dependencies

Dependencies allow the user and installation system to require other packages.
There is 2 kinds of dependencies:

- build dependencies in `pyproject.toml`:
  ```toml
  [build-system]
  requires = ["setuptools", "wheel", "cython"]
  ```
- runtime dependencies in `setup.cfg`
  ```cfg
  [options]
  packages = find:
  python_requires = >=3.6
  install_requires = 
	numpy
    h5py
  ```

Dependencies are propagated to binary packages (wheels, Debian...)

Dependency syntax: [PEP508](https://www.python.org/dev/peps/pep-0508/)

#### Optional requirements

`setup.cfg`'s `[options.extras_require]` section allows to define optional dependencies:

```cfg
[options.extras_require]
dev =
    pytest
	sphinx
```

It is then possible to install those extra dependencies with:

```
pip install package[dev]
```

#### `setup.cfg` resources

- [[metadata] fields](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html#metadata)
- [[options] fields](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html#options)
- [List of keywords](https://setuptools.pypa.io/en/latest/references/keywords.html)

## Distribution

The central registration point is the Python Package Index [PyPI](https://pypi.org/).

![](PyPI.png)


Use the `twine` package to publish to [pypi.org](https://pypi.org/account/register/):

- First, create an account on [pypi.org](https://pypi.org/account/register/) (or the test instance: [test.pypi.org](https://test.pypi.org/account/register/)) 
- Generate the packages you want to provide (check the version number, and tag it in git):

  `python -m build  # or python setup.py sdist bdist_wheel`
- Upload the project with `twine` (`pip install twine`):

  `twine upload [--sign] dist/*`

  or for [test.pypi.org](https://test.pypi.org): `twine upload [--sign] --repository-url https://test.pypi.org/legacy/ dist/*`
- Install from [pypi.org](https://pypi.org/): `pip install package`

  or for [test.pypi.org](https://test.pypi.org): `pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple package`

## Advanced features

### `requirements.txt` vs `setup.cfg`

It may look contradictory to define dependencies at different places but it is not https://packaging.python.org/discussions/install-requires-vs-requirements/.

* `setup.cfg` provides abstract minimal dependency requirements (e.g., `numpy`)
* `requirements.txt` provides concrete implementation
  (with hard coded versions and URL to download wheels from).
  This provides a way to specify an environment: `numpy==1.12.0`

  Usage: `pip install -r requirements.txt`

### Entry points

Allows to define functions that are available as commands once installed.
It provides a cross-platform automatic script creation mechanism (i.e., produces .exe files on Windows and scripts on other OS).

`setup.cfg`:
```cfg
[options.entry_points]
console_scripts =
    package-script = package:main
gui_scripts =
    package-gui = package.gui:main
```

Will create a `package-script` and a `package-gui` command-line scripts during installation.

### `MANIFEST.in`: Source package content

A default set of files (`*.py`, `LICENSE`, `README.md`) is included in source packages produced by:
```
python -m build  # or python setup.py sdist
```

It is possible to include additional files by declaring them in a `MANIFEST.in` file at the project's top-level:

```
include CONTRIBUTE.txt
recursive-include package *.dat
graft example
```

See [documentation](https://packaging.python.org/guides/using-manifest-in/)

### Additional resources

Install non-Python files within the package (e.g., reference data).

#### Automatic

Add needed files to `MANIFEST.in` and add the following to `setup.cfg`:
```cfg
[options]
include_package_data = True
```

#### Manual

```cfg
[options]
package_data =
    * = *.dat  # * Applies to all packages
    package.subpackage = *.dat 
```

See [Data Files Support](https://setuptools.pypa.io/en/latest/userguide/datafiles.html) documentation.

### Alternative package folder

Change the folder storing package source:
```
project/
  ...
  src/
    package/
      __init__.py
      ...
```

`setup.cfg`:
```cfg
[options]
package_dir=
    =src
```

### `setup.cfg` vs. `setup.py`

`setup.cfg`:
```cfg
[metadata]
name = package
version = 0.0.1

[options]
packages = find:
```
is equivalent to `setup.py`:
```python
import setuptools

setuptools.setup(
    name="package",
    version="0.0.1",
    packages=setuptools.find_packages(),
)
```

As of today, `setup.cfg` is the recommended way, but `setup.py` is fully working.

### Compiled extensions

It is possible to compile modules written in C, C++, Cython as part of the build process.

```python
from setuptools.extension import Extension

setup(
    ext_modules=[
        Extension('package.cmodule', ['package/cmodule.c'])],
        Extension('package.cythonmodule', ['package/cmodule.pyx'])
    ],
)
```

This adds the requirement of having the proper compiler available and put a lot more constraint on packaging and distribution (e.g., one wheel per operating system and per architecture built with a specific environment).

### Advanced setup.py: numpy.distutils

[numpy.distutils](https://docs.scipy.org/doc/numpy/reference/distutils.html) provides a way to use a hierarchy of `setup.py`:

```
project/
  setup.py  -> Effective setup.py
  package/
    __init__.py
    setup.py      -> Handles main package build
    subpackage/
      __init__.py
      setup.py    -> Handles subpackage build
```

- Pros: Each sub-package is embedding its own build.
- Cons: `numpy` becomes a build dependency.

This is based on the [Configuration class](https://docs.scipy.org/doc/numpy/reference/distutils.html#numpy.distutils.misc_util.Configuration).

## A word on automated deployment

It is possible to automate the release process with "continous integration" services:

- Ease version number handling and git tag: [bump2version](https://pypi.org/project/bump2version/)
- Ease generation of compiled wheels: [cibuildwheel](https://cibuildwheel.readthedocs.io/en/stable/)
- Possible to set continuous integration service to publish release on pypi.org (using a token from pypi): [Example here](https://github.com/silx-kit/h5grove/blob/f44bc762ebcf02e1db2e51e442552c469f95f586/.github/workflows/release.yml#L39-L46)

## A word on packages

### Wheels: [PEP427](https://www.python.org/dev/peps/pep-0427/)


Wheels are the current standard of Python distribution through [pypi.org](https://pypi.org/).

#### Advantages

1. Avoids arbitrary code execution for installation (no `setup.py` executed).
1. Does not require a compiler on the user side for binary extensions.
1. Faster installation, especially for binary extensions.
1. Creates `*.pyc` files at installation, matching the Python interpreter used.
1. More consistent installs across platforms and machines.

Wheels provide binary packages and a decent installer (`pip`).
It is a very convenient way to install up-to-date versions.

#### Pitfalls

- For compiled extension, a specific compilation environment is required (e.g., [manylinux](https://github.com/pypa/manylinux) docker under Linux). See [Building binary extensions doc](https://packaging.python.org/guides/packaging-binary-extensions/#building-binary-extensions)).
- External shared library needs to be incorporated in the wheel.
  You can use utility software to check against which libraries your package is linked :

  - macOS: [delocate](https://github.com/matthew-brett/delocate)
  - Windows: [depends](http://www.dependencywalker.com/)
  - Linux: ldd, [auditwheel](https://github.com/pypa/auditwheel)

### Debian/Ubuntu packages

Useful tools to create Debian packages from Python packages:

- [stdeb](https://pypi.python.org/pypi/stdeb/): Takes a source Python project as input.
- [wheel2deb](https://pypi.org/project/wheel2deb/): Takes wheels as input.

Might need to edit generated Debian packaging configuration to change dependencies.

### Fat binaries

Standalone self-contained applications or installers.

- Include Python interpreter and all dependencies.
- Fits Windows and macOS application distribution, as unlike Linux they lack a dependency management tool.

Beware:

- Fat binaries are fat (~150 Mb for projects involving GUIs).
- You are redistributing many other people's work, so take care about licenses.

#### Freezing

There is a number of tools to 'freeze' a Python application for distribution from an installation on a computer.

Principle:

- Analyze a script to find its dependencies (i.e., its imports).
- Collect all dependencies and python interpreter in a directory.
- Add a launcher and eventually bundle everything in a single file or installer.

#### Freezing issues

- Those tools relies on rules specific to each package (`matplotlib`, `numpy`) which needs to be updated when packages evolve.
- Analysis can miss some hidden imports.
- All runtime dependencies must be included (including external libraries wrapped by Python packages).
- Data files cannot be guessed and need to be explicitly added.

You must make sure it is stand-alone and includes everything required.
Test the result on a different computer than the one used for packaging.

#### Tools

[PyInstaller](http://www.pyinstaller.org/): Cross-platform

But also
[cx_Freeze](http://cx-freeze.readthedocs.org/) (cross-platform),
[py2app](https://pythonhosted.org/py2app/) (macOS),
[pynsist](https://pypi.python.org/pypi/pynsist) (Windows),
[py2exe](https://pypi.python.org/pypi/py2exe/) (Windows),
[pex](https://github.com/pantsbuild/pex) (Linux, macOS)

On Windows, you can create an installer with a tool such as [NSIS](http://nsis.sourceforge.net/).